In [ ]:
# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import folium  # for geospatial visualization

In [ ]:
# Load and Visualize Sample Data

In [ ]:
# Example tabular climate data (NOAA or Kaggle subset)
climate_data = pd.read_csv("climate_data_subset.csv")
print(climate_data.head())

# Basic visualization of climate trends
plt.figure(figsize=(10,5))
sns.lineplot(data=climate_data, x='date', y='temperature')
plt.title("Sample Temperature Trend")
plt.xlabel("Date")
plt.ylabel("Temperature (°C)")
plt.show()